#### To test: 
1. Create a folder ~/data/luna16/
2. Create a folder ~/data/luna16/hdf5
    -Under this copy UNET_weights_H5.h5 (download from google drive)
3. Create a folder ~/data/luna16/hdf5/HO5 
    -Under this copy 64x64x64-patch-withdiam_sample.hdf5

In [1]:
import numpy as np
import h5py
import pandas as pd
import keras

from ipywidgets import interact

from UNET_utils import *
from matplotlib import pyplot as plt
%matplotlib inline

/home/tony/anaconda3/envs/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
HOLDOUT = 5
TOP_DIR = "/home/tony"
HO_dir = 'hdf5/HO{}/'.format(HOLDOUT)
data_dir = '{}/data/luna16/'.format(TOP_DIR)
model_wghts = 'hdf5/UNET_weights_modelB_H{}.h5'.format(HOLDOUT)

In [3]:
def model_create_loadWghts_Model_B():
   
    input_shape=(None, None, None, 1)
    model = unet3D_modelB(input_shape, use_upsampling=True)

    model.load_weights(data_dir + model_wghts)

    return model

In [4]:
model = model_create_loadWghts_Model_B() 

3D U-Net Segmentation


In [5]:
hdf = h5py.File("{}{}/64x64x64-patch-withdiam_sample.hdf5".format(data_dir, HO_dir), "r")

In [6]:
list(hdf.keys())

['centroid', 'diameter_label', 'input', 'notrain', 'output', 'subsets', 'uuid']

In [7]:
class1 = np.where(hdf["output"][:,0]==1)[0]  # Where are class 1
class0 = np.where(hdf["output"][:,0]==0)[0]  # Where are class 1

In [8]:
class1[0]

28

In [9]:
print("Class 1")

class1idx = 0

# Choose one of the class 1 ROIs

img1 = hdf["input"][class1[class1idx],:].reshape(64,64,64,1)  


predict_array = model.predict(np.expand_dims(img1,0), verbose=2)

prediction_mask1 = predict_array[0]
prediction_featuremaps1 = predict_array[1]


def displaySlice(sliceNo):
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img1[:,:,sliceNo,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask1[0,:,:,sliceNo,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img1[:,:,sliceNo,0], cmap="bone")
    plt.imshow(prediction_mask1[0,:,:,sliceNo,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img1[sliceNo,:,:,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask1[0,sliceNo,:,:,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img1[sliceNo,:,:,0], cmap="bone")
    plt.imshow(prediction_mask1[0,sliceNo,:,:,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
interact(displaySlice, sliceNo=(1,img1.shape[2],1)); 

print(hdf["uuid"][0])

Class 1


A Jupyter Widget

[b'1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405']


In [10]:
print("Class 0")
class0idx = 50

# Choose one of the class 0 ROIs

img0 = hdf["input"][class0[class0idx],:].reshape(64,64,64,1)  


predict_array = model.predict(np.expand_dims(img0,0), verbose=2)

prediction_mask0 = predict_array[0]
prediction_featuremaps0 = predict_array[1]


def displaySlice(sliceNo):
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img0[:,:,sliceNo,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask0[0,:,:,sliceNo,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img0[:,:,sliceNo,0], cmap="bone")
    plt.imshow(prediction_mask0[0,:,:,sliceNo,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,3,1)
    plt.imshow(img0[sliceNo,:,:,0], cmap="bone")
    plt.title("Image")

    plt.subplot(1,3,2)
    plt.imshow(prediction_mask0[0,sliceNo,:,:,0], cmap="bone")
    plt.title("Tumor Prediction")

    plt.subplot(1,3,3)
    plt.imshow(img0[sliceNo,:,:,0], cmap="bone")
    plt.imshow(prediction_mask0[0,sliceNo,:,:,0], alpha=0.5, cmap="Reds")
    plt.title("Overlay")
    
    
    
interact(displaySlice, sliceNo=(1,img0.shape[2],1)); 

print(hdf["uuid"][0])

Class 0


A Jupyter Widget

[b'1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016233746780170740405']


In [11]:
#hdf.close()

In [12]:
img1.shape

(64, 64, 64, 1)

In [13]:
img0.shape

(64, 64, 64, 1)

In [14]:
prediction_mask0.shape

(1, 64, 64, 64, 1)

In [15]:
prediction_mask1.shape

(1, 64, 64, 64, 1)

In [19]:
np.savez_compressed('for_anil.npz', img1=img1, img0=img0, 
                    prediction_mask1=prediction_mask1, prediction_mask0=prediction_mask0, 
                   prediction_featuremaps0=prediction_featuremaps0, prediction_featuremaps1= prediction_featuremaps1)